In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
import register_ as reg
import algo_cooling as algo
import algo_dmrg 

import quf
import time
import numpy as np

/Users/rezah/envs/genpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-24 11:00:38,843	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=True)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=True)

to_backend_ = algo.backend_numpy( dtype = "complex128")

opt = algo.opt_(progbar=False)

In [3]:
Lx, Ly = (4, 4)
peps = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=4, seed=666, dtype="complex128")
peps_ = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=3, seed=666, dtype="complex128")

# peps.equalize_norms_(4.0)


pepsh = peps_.H
pepsh.add_tag('BRA')
peps.add_tag('KET')
norm = pepsh & peps

norm.apply_to_arrays(to_backend)


In [4]:
# peps.outer_inds()

In [5]:
# norm.draw(["BRA", "KET", "PEPO"])

In [6]:
norm_ = norm.contract(all, optimize=opt)
norm = norm * (norm_**-1.)

In [7]:
# norm_

In [8]:
norm_ = norm.contract(all, optimize=opt)
norm_

tensor(1.0000+4.1633e-16j, dtype=torch.complex128, grad_fn=<MulBackward0>)

In [9]:
track_ = algo.Trck_boundary(peps, opt=opt, cutoffs=1.e-12, chi=60,
                            to_backend=to_backend, to_backend_=to_backend_)

mps_b = track_.mps_b
mpo_b = track_.mpo_b

In [10]:
mpo_b

{'Y0_l': MatrixProductState(tensors=4, indices=11, L=4, max_bond=60),
 'Y1_l': MatrixProductState(tensors=4, indices=11, L=4, max_bond=60),
 'Y2_l': MatrixProductState(tensors=4, indices=11, L=4, max_bond=60),
 'Y0_r': MatrixProductState(tensors=4, indices=11, L=4, max_bond=60),
 'Y1_r': MatrixProductState(tensors=4, indices=11, L=4, max_bond=60),
 'Y2_r': MatrixProductState(tensors=4, indices=11, L=4, max_bond=60)}

In [11]:
# mps_b

In [12]:
# norm_ = norm.contract(all, optimize=opt)
# norm_

In [13]:
# norm.exponent

In [14]:
cut_tag_id = "Y{}"
site_tag_id = "X{}"
norm_l = []

for count in range(Ly-1):
     
    print("Y", count, site_tag_id.format(count))
    if count == 0:
        p = mps_b[cut_tag_id.format(count) + "_l"]
        tn = norm.select(cut_tag_id.format(count), "any")
    else:
        tn = norm.select(cut_tag_id.format(count), "any") | mps_b[cut_tag_id.format(count-1)+ "_l"]
        p = mps_b[cut_tag_id.format(count)+ "_l"]


    fit = algo_dmrg.FIT(tn.copy(), p=p, site_tag_id="I{}", opt=opt, re_tag=True)
    # fit.visual(figsize=(8,8), show_inds="bond-size", tags_=[])

    #start_time = time.time()
    #fit.run(n_iter=1, verbose=False)
    #print(f"--- %s no-reuse ---" % (time.time() - start_time))
    start_time = time.time()
    fit.run_eff(n_iter=1, verbose=False)
    print(f"--- %s reuse ---" % (time.time() - start_time))
    start_time = time.time()
    value = fit.p.norm()
    norm_l.append(value)

    #print(quf.fidel_mps(tn, fit.p))
    #print(f"--- %s fidel ---" % (time.time() - start_time))
    
    mps_b[cut_tag_id.format(count)+ "_l"] = fit.p / value

tn & p contains different outer inds 


Y 0 X0


KeyError: '_eaa3c1AAAAw'

In [ ]:

res = mps_b[cut_tag_id.format(Ly-2)+ "_l"] & norm.select(cut_tag_id.format(Ly-1), "any")
res.draw(show_inds="bond-size")
print( res.contract(all, optimize=opt)*np.prod(norm_l) )

In [ ]:
%%time
norm_l = []
cut_tag_id = "Y{}"
site_tag_id = "X{}"
for count in range(Ly-1):
     
    print("Y", count, site_tag_id.format(count))
    if count == 0:
        p = mpo_b[cut_tag_id.format(count) + "_l"]
        tn = norm.select(cut_tag_id.format(count), "any")
    else:
        tn = norm.select(cut_tag_id.format(count), "any") | mpo_b[cut_tag_id.format(count-1)+ "_l"]
        p = mpo_b[cut_tag_id.format(count)+ "_l"]


    fit = algo_dmrg.FIT(tn, p=p, site_tag_id=site_tag_id, opt=opt, re_tag=False)
    # fit.visual(figsize=(8,8), show_inds="bond-size", tags_=[])

    #start_time = time.time()
    #fit.run(n_iter=1, verbose=False)
    #print(f"--- %s no-reuse ---" % (time.time() - start_time))
    
    start_time = time.time()
    fit.run_eff(n_iter=1, verbose=False)
    print(f"--- %s reuse ---" % (time.time() - start_time))
    #start_time = time.time()
    #print("fidel=", algo.fidel_mps(tn, fit.p))
    #print(f"--- %s fidel ---" % (time.time() - start_time))
    #print("norm", fit.p.norm())
    value = fit.p.norm()
    norm_l.append(value)
    
    mpo_b[cut_tag_id.format(count)+ "_l"] = fit.p / value

In [ ]:

res = mpo_b[cut_tag_id.format(Ly-2)+ "_l"] & norm.select(cut_tag_id.format(Ly-1), "any")
res.draw(show_inds="bond-size")
print( res.contract(all, optimize=opt)*np.prod(norm_l) )

In [ ]:
%%time
val_0 = norm.contract_boundary(max_bond=50, 
                                   mode="mps",#     "full-bond", #   'mps',
                                   #sequence=["ymin"],
                                   final_contract_opts={"optimize": opt}, 
                                   cutoff=1e-14,
                                   progbar=True,
                                   layer_tags=['KET', 'BRA'],
                                   max_separation=0,
                                  )
val_0

In [ ]:
def apply_hyperoptimized_compressed(tn, copt, chi, output_inds=None, tree_gauge_distance=4, progbar=False, 
                                    cutoff=1.e-12, equalize_norms=False):
    
    tn.full_simplify_(seq='R', split_method='svd', inplace=True)
    
    tree = tn.contraction_tree(copt)
    tn_ = tn.copy()
    
    flops = tree.contraction_cost(log=10)
    peak = tree.peak_size(log=2)
    
    tn_.contract_compressed_(
        optimize=tree,
        output_inds=output_inds,
        max_bond=chi,
        tree_gauge_distance=tree_gauge_distance,
        equalize_norms=equalize_norms,
        cutoff=cutoff,
        progbar=progbar,
    )
    return tn_, (flops, peak)


In [ ]:
%%time
chi = 50
copt = ctg.ReusableHyperCompressedOptimizer(
    chi,
    max_repeats=512,
    minimize="combo-compressed",
    progbar=True,
    # # save paths to disk:
    # directory="cash"
)

overlap, (flops, peak) = apply_hyperoptimized_compressed(norm, copt, chi, cutoff=1.e-12, progbar=True)
# main, exp=(overlap.contract(all), overlap.exponent)
val_0 = overlap^all 
print(val_0)
